In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math as mt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from hangul_utils import split_syllables, join_jamos
from tqdm.auto import tqdm
import re
from eunjeon import Mecab
from hanspell import spell_checker

mec = Mecab()


##높임말 리스트. 다음에 해당하면 높임말로 판단한다.
H_LIST = ['니다', '니까', '요', '시오', '죠']

##높임말로 변환할 때 중성끼리 합치는 경우가 있다. 그러한 경우 합치기 위한 사전이다. 
con_dict = [
    
    ['ㅏㅣ','ㅐ'], ['ㅑㅣ','ㅒ'], ['ㅓㅣ','ㅔ'],
    ['ㅕㅣ','ㅖ'], ['ㅗㅣ','ㅚ'], ['ㅗㅐ','ㅙ'],
    ['ㅜㅓ','ㅝ'], ['ㅜㅔ','ㅞ'], ['ㅡㅣ','ㅢ'],
    ['ㅣㅏ','ㅑ'], ['ㅣㅓ','ㅕ'], ['ㅣㅗ','ㅛ'],
    ['ㅣㅜ','ㅠ'], ['ㅡㅓ','ㅓ'], ['ㅗㅏ','ㅘ']
    
]

####high -> low Dictionary####

##EF(종결어미): 형태소 분류 시 종결어미가 단독으로 분석 되는 경우
EF_ONLY = [
    
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']], #ex)'습니다'-> '다'
    [['ㄷ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']],
    [['ㄹ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']],
    [['ㄴ','ㅏ','ㅇ','ㅛ'],['ㄴㅏ']],
    [['ㄷ','ㅐ','ㅇ','ㅛ'],['ㄷ','ㅐ']],
    [['ㅇ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㄴ','ㅔ','ㅇ','ㅛ'],['ㄷ','ㅏ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ'],['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ']],
    [['ㅈ','ㅛ'],['ㅈ','ㅣ']],
    [['ㅇ','ㅏ','ㅇ','ㅛ'],['ㅇ','ㅏ']],
    [['ㅇ','ㅓ','ㅇ','ㅛ'],['ㅇ','ㅓ']],
    [['ㅇ','ㅡ','ㅅ','ㅔ','ㅇ','ㅛ'],['specialx']],
    [['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ','ㅇ','ㅛ'],['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ']],
    #[['ㅎ','ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㅎ','ㅇ','ㅏ']],
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㅇ','ㅓ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ'],['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ']],
    [['ㅇ','ㅖ','ㅇ','ㅛ'],['ㄷㅏ']],
    [['ㅇ','ㅛ'],['']],
    
]

#EP(선어말어미)+EF(종결어미)로 형태소가 분석될 경우
EP_EF = [
    
    [['ㅅ','ㅔ','ㅇ','ㅛ'],['specialx']]
    #-시오 family
    ,[['ㅡ','ㅅ','ㅣ','ㅂ','ㅅ','ㅣ','ㅇ','ㅗ'],['ㅓ']]
    ,[['ㅅ','ㅣ','ㅂ','ㅅ','ㅣ','ㅇ','ㅗ'],['ㄹ', 'ㅏ']],
    [['ㅅ','ㅣ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㄴ','ㅣ']],
    [['ㅅ','ㅣ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ'],['ㄹ','ㄹ','ㅐ']],
]

#VCP+EF
VCP_EF = [
    
    [['ㅇ','ㅣ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']],
    [['ㅇ','ㅣ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['-special']],
    [['ㅇ','ㅣ','ㄴ','ㄱ','ㅏ','ㅇ','ㅛ'],['-special']],
    [['ㅈ','ㅛ'],['ㅇ','ㅑ']],
    [['ㅇ','ㅣ','ㅇ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅣ','ㅇ','ㅑ']],
    [['ㅇ','ㅖ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㄱ','ㅓ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㄹ','ㅏ','ㄷ','ㅓ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ'],['ㄹ','ㅏ','ㄷ','ㅓ','ㄴ','ㄷ','ㅔ']],
    [['ㄹ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']],
    [['ㄷ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']],
    
]

VCN_EF = [
    
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㅇ','ㅑ']],
    
]

#XSA+EF
XSA_EF = [
    [['ㅎ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㅎ','ㅏ','ㄷ','ㅏ']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']],
    [['ㅇ','ㅛ'],['']]
]

#XSV+EF
XSV_EF = [
    [['ㅎ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㅎ','ㅐ']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['']],
    [['ㅐ','ㅇ','ㅛ'],['ㅐ']],
]

#alpha+EF: 위의 경우를 제외한 나머지 경우를 의미한다. 
A_EF = [
    [['ㄱ','ㅓ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄴ','ㄷ','ㅏ']],
    [['ㄱ','ㅖ','ㅅ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅣ','ㅆ','ㅇ','ㅓ']],
    [['ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㄴ','ㅏ']],
    [['ㅇ','ㅛ'],['']],
    [['ㄷ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']]
    
    
]

##높임표현이 어말어미가 아닌 경우##
# - 기존의 변환 방식이었다. 이제는 쓰지는 않지만, 코드의 오류가 발생할 가능성이 있기에 지우지 않았다. 따로 참고할 필요 없음.
EF = [
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']],
    [[' ','ㅈ','ㅓ','ㄴ',' '],['ㄴ','ㅏ','_','ㄴ','ㅡ','ㄴ']],
    [['ㅇ','ㅓ','_','ㅇ','ㅛ','_'],['ㄷ','ㅏ','_']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄴ','ㄷ','ㅏ']],
    [['ㅇ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㅇ','ㅛ'],['']]
    #,[['ㅅ','ㅔ'],['ㅝ'],['ㅘ']]
]

NP = [
    
    [['ㅈ','ㅓ','ㄴ'],['special-']],
    [['ㅈ','ㅓ'],['ㄴ','ㅏ']]
    
]

# 보조사
JX = [
    [['ㅇ','ㅣ','ㅇ','ㅛ'],['ㅇ','ㅣ','ㅇ','ㅑ']],
    [['ㅇ','ㅛ'],['']]
]

JKB_JX = [
    [['ㅇ','ㅛ'],['specialx']]
]



VX = [
    
    [['ㅈ','ㅜ'],['ㅈ','ㅜ'],['ㅈ','ㅝ']]
    
]

VV = [
    
    [['ㅇ','ㅗ'],['ㅇ','ㅗ'],['ㅇ','ㅘ']],
    [['ㅈ','ㅜ'],['ㅈ','ㅜ'],['ㅈ','ㅝ']],
    [['ㅎ','ㅏ'],['ㅎ','ㅏ'],['ㅎ','ㅐ']]
    
]

XR = [
    [['ㅈ','ㅚ','ㅅ','ㅗ','ㅇ'],['ㅁ','ㅣ','ㅇ','ㅏ','ㄴ'] ]
]

NNG = [
    [['ㅈ','ㅏ','ㅇ','ㅛ'],['ㅈ','ㅏ']],
    #[['ㄱ','ㅏ','ㅁ','ㅅ','ㅏ'],['ㅈ','ㅏ']],
]

NNP =[
    [[],[]]
]

EC = [
    
    [['ㅇ','ㅛ'],['ㄷ','ㅏ']]
    
]

EXC_deal_1 = [
    
    ['ㅅㅔㅇㅛ', ['ㅏ','ㅣ'],['ㅜ','ㅓ'],['ㄹㅡ','ㄹㄹㅓ'],['ㅡ','ㅓ'],['ㅣ','ㅓ']],
]

EXC_deal_2 = [
    
    ['ㅈㅓㄴ','ㅈㅓㄴ','ㄴㅏㄴ'],
]

EXC_deal_3 = [
    
    ['ㄴㅏㅇㅛ',[['ㅆ', 'ㅇㅓ']],'아'],
    ['ㅅㅡㅂㄴㅣㄷㅏ',[['ㅆ','ㄷㅏ'],['ㅎ','ㄷㅏ'],['ㅄ','다'],['ㅂ','ㄷㅏ'],['ㅌ','ㄷㅏ'],['ㅍ','ㄷㅏ'],['ㄵ','ㄷㅏ']],'ㄴㅡㄴㄷㅏ'],
    ['ㅇㅣㅂㄴㅣㄲㅏ',[['ㄹ','ㅇㅣㅇㅑ'],['ㅂ','ㅇㅣㅇㅑ'],['ㅅ','ㅇㅣㅇㅑ'],['ㄷ','ㅇㅣㅇㅑ'],['ㅁ','ㅇㅣㅇㅑ'],['ㄴ','ㅇㅣㅇㅑ'],['ㄱ','ㅇㅣㅇㅑ'],['ㅄ','ㅇㅣㅇㅑ'],['ㅇ','ㅇㅣㅇㅑ'],['ㅈㅗ','ㅇㅕ'],['ㅂㅗ','ㅇㅕ']],'ㅇㅑ'],
    ['ㅇㅣㅂㄴㅣㄷㅏ',[['ㄹ','ㅇㅣㄷㅏ'],['ㅂ','ㅇㅣㄷㅏ'],['ㅅ','ㅇㅣㄷㅏ'],['ㄷ','ㅇㅣㄷㅏ'],['ㅁ','ㅇㅣㄷㅏ'],['ㄴ','ㅇㅣㄷㅏ'],['ㄱ','ㅇㅣㄷㅏ'],['ㅄ','ㅇㅣㄷㅏ'],['ㅇ','ㅇㅣㄷㅏ'],['ㅈㅗ','ㅇㅣㄴㄷㅏ'],['ㅂㅗ','ㅇㅣㄴㄷㅏ']],'ㄷㅏ'],
    ['ㅇㅣㄴㄱㅏㅇㅛ',[['ㄱ','ㅇㅣㅇㅑ'],['ㄴ','ㅇㅣㅇㅑ'],['ㄷ','ㅇㅣㅇㅑ'],['ㄹ','ㅇㅣㅇㅑ'],['ㅁ','ㅇㅣㅇㅑ'],['ㅂ','ㅇㅣㅇㅑ'],['ㅇ','ㅇㅣㅇㅑ'],['ㅄ','ㅇㅣㅇㅑ'],['ㅀ','ㅇㅣㅇㅑ']],'ㅇㅑ'],
    ['ㄷㅏㅂㄴㅣㄷㅏ',[['','']],'ㄷㅏㄴㄷㅏ'],
    ['ㄹㅏㅂㄴㅣㄷㅏ',[['','']],'ㄹㅏㄴㄷㅏ'],
    
]

EXC_deal_4 = [

    ['ㅇㅛ','',[['ㅇㅔ','ㅇㅣ','ㄷㅏ'],''],''],
    ['ㅅㅔㅇㅛ', 'ㄷㅏ', [['ㅏ','ㅏ','ㅣ'],['ㅜㄹㅡ','ㅜㄹ','ㄹㅓ'],['ㅜ','ㅜ','ㅓ'],['ㅡ','ㅡ','ㅓ'],['ㅐ','ㅐ',''],['ㅣ','ㅣ','ㅓ'],'ㅇㅓ'],'ㅇㅑ'],
    ['ㅇㅡㅅㅔㅇㅛ','ㄷㅏ',[['ㄴㅗㅎ','ㄴㅗㅏ',''],''],'ㅇㅏ'],

]



EXC_tags = [
    
    'EP',
    'EF'
    
]

UNKNOWN = [
    [['ㄷ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅡ','ㄴ','ㄷ','ㅏ']],
]

####End of high -> low Dictionary####


#### low -> high Dictionary####
## 사전의 순서가 중요하다. 순서를 임의로 바꾸지 말 것
## EF solo. EF 앞에 형태소가 결합형이 아닌 홀로 분석 될 경우 ex) /EP, /EF

EF_ONLY_4S = [
    # -다면서 /EF
    [['ㄷ','ㅏ','ㅁ','ㅕ','ㄴ','ㅅ','ㅓ'],['ㄷ','ㅏ','ㅁ','ㅕ','ㄴ','ㅅ','ㅓ','ㅇ','ㅛ']],
    # -고 /EF
    [['ㄹ','ㅏ','ㄱ','ㅗ'],['ㄹ','ㅏ','ㄱ','ㅗ','ㅇ','ㅛ']], ##-라고-> 라고요
    [['ㄷ','ㅓ','ㄹ','ㅏ','ㄱ','ㅗ'],['ㄷ','ㅓ','ㄹ','ㅏ','ㄱ','ㅗ','ㅇ','ㅛ']],
    [['ㄷ','ㅏ','ㄱ','ㅗ'],['ㄷ','ㅏ','ㄱ','ㅗ','ㅇ','ㅛ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅏ'],['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㄴ','ㄷ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    #-어서다 /EF
    [['ㅇ','ㅓ','ㅅ','ㅓ','ㄷ','ㅏ'],['ㅇ','ㅓ','ㅅ','ㅓ','ㅇ','ㅣ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㄷ','ㅏ'],['-special']],
    [['ㅇ','ㅓ','ㄹ','ㅏ'],['ㅇ','ㅡ','ㅅ','ㅔ','ㅇ','ㅛ']], #add jh_1115
    [['ㅇ','ㅓ'],['ㅇ','ㅓ','ㅇ','ㅛ']],
    #'-라' /EF
    [['ㄷ','ㅓ','ㄹ','ㅏ'],['ㄷ','ㅓ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㅇ','ㅏ','ㄹ','ㅏ'],['ㅇ','ㅏ','ㅇ','ㅛ']],
    [['ㄹ','ㅣ','ㄹ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']], ##'-리라'는 아직 해결하지 못하였다. 
    [['ㄹ','ㅏ'],['ㅅ','ㅔ','ㅇ','ㅛ']],
    
    
    # -데 /EF
    [['ㅇ','ㅡ','ㄴ','ㄷ','ㅔ'],['ㅇ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ'],['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㄷ','ㅓ','ㄴ','ㄷ','ㅔ'],['ㄷ','ㅓ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    # -걸 /EF
    [['ㄴ','ㅡ','ㄴ','ㄱ','ㅓ','ㄹ'],['ㄴ','ㅡ','ㄴ','ㄱ','ㅓ','ㄹ','ㅇ','ㅛ']],
    
    [['ㅈ','ㅏ','ㄶ','ㅇ','ㅏ'],['ㅈ','ㅏ','ㄶ','ㅇ','ㅏ','ㅇ','ㅛ']], ## -잖아 -> -잖아요
    
    #-오 /EF
    [['ㅇ','ㅗ'],['ㅅ','ㅔ','ㅇ','ㅛ']],
    #-니 /EF
    [['ㄴ','ㅣ'],['ㄴ','ㅏ','ㅇ','ㅛ']],
    #-자 /EF
    [['ㅈ','ㅏ'],['ㅅ','ㅔ','ㅇ','ㅛ']],
    #-래 /EF
    [['ㅇ''ㅡ','ㄹ','ㄹ','ㅐ'],['ㅇ''ㅡ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ']],
    [['ㄹ','ㅐ'],['ㄹ','ㅐ','ㅇ','ㅛ']],
    
    
    # -거든 /EF
    [['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ'],['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ','ㅇ','ㅛ']],
    
    #-마 /EF
    [['ㅁ','ㅏ'],['ㅁ','ㅏ','ㅇ','ㅛ']],
    
    #-아 /EF
    [['ㅇ','ㅏ'],['ㅇ','ㅏ','ㅇ','ㅛ']],
    #-지/EF
    [['ㅈ','ㅣ'],['ㅈ','ㅛ']],
    
    #-나 /EF
    [['ㄴ','ㅏ'],['ㄴ','ㅏ','ㅇ','ㅛ']],

    #[['ㄴ','ㅡ','ㄴ','ㄱ','ㅏ'],['-special']],
    #-는가 /EF
    [['ㄴ','ㅡ','ㄴ','ㄱ','ㅏ'],['ㄴ','ㅏ','ㅇ','ㅛ']],
    #-던가 /EF
    [['ㄷ','ㅓ','ㄴ','ㄱ','ㅏ'],['ㄷ','ㅓ','ㄴ','ㄱ','ㅏ','ㅇ','ㅛ']],
    #-을까 /EF
    [['ㅇ''ㅡ','ㄹ','ㄲ','ㅏ'],['ㅇ''ㅡ','ㄹ','ㄲ','ㅏ','ㅇ','ㅛ']],
    
    # -ㅏ /EF
    [['ㅏ'],['ㅏ','ㅅ','ㅔ','ㅇ','ㅛ']],
    # -ㅔ /EF
    [['ㅔ'],['ㅔ','ㅇ','ㅛ']],
    
    
    
    
]
## EF 결합형
## EF 앞에 형태소가 결합형인 경우 앞의 형태소도 고려해서 처리해야 한다. ex) /EP, /EF
EF_ONLY_4C = [
    
    [['ㄷ','ㅏ'],['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㅇ','ㅓ'],['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㅇ','ㅓ','ㄸ','ㅐ'],['ㅇ','ㅓ','ㄸ','ㅐ','ㅇ','ㅛ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ'],['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    
    # -걸 /EF
    [['ㄴ','ㅡ','ㄴ','ㄱ','ㅓ','ㄹ'],['ㄴ','ㅡ','ㄴ','ㄱ','ㅓ','ㄹ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['-special']],
    # -니 /EF
    [['ㄴ','ㅣ'],['ㅇ','ㅓ','ㅇ','ㅛ']],
    # -거든 /EF
    [['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ'],['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ','ㅇ','ㅛ']],
    [['ㅈ','ㅣ'],['ㅈ','ㅛ']],
    #[['ㄴ','ㅡ','ㄴ','ㄱ','ㅏ'],['-special']],
    [['ㄴ','ㅡ','ㄴ','ㄱ','ㅏ'],['ㄴ','ㅏ','ㅇ','ㅛ']],
    [['ㄴ','ㅏ'],['ㄴ','ㅏ','ㅇ','ㅛ']],
    
]

EP_EF_4 =[
    #[['ㄴ','ㅔ'],['ㄱ','ㅓ','ㄹ','ㄲ','ㅏ','ㅇ','ㅛ']],
    [['ㄴ','ㅔ'],['ㄴ','ㅔ','ㅇ','ㅛ']],
]

#/VCP+EF
VCP_EF_4 = [
    #[['ㄱ','ㅓ','ㄹ','ㄲ','ㅏ'],['ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㄱ','ㅓ','ㄹ','ㄲ','ㅏ'],['ㄱ','ㅓ','ㄹ','ㄲ','ㅏ','ㅇ','ㅛ']], ##bug check
    [['ㄱ','ㅓ','ㄹ'],['ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㅇ','ㅑ'],['ㅇ','ㅔ','ㅇ','ㅛ']],
    [['ㄹ','ㅐ'],['ㄹ','ㅐ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['ㅇ','ㅔ','ㅇ','ㅛ']], #add jh_1116
    [['ㄷ','ㅔ'],['ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㄷ','ㅏ'],['-special']]
    
]

#/VA+EF added code_JH
VA_EF_4 = [
    [['ㅇ','ㅓ','ㄸ','ㅐ'],['ㅇ','ㅓ','ㄸ','ㅐ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['-special']],
    [['ㅕ'],['ㅕ','ㅇ','ㅛ']],
]

## /VCP, /EF
EF_AFTER_VCP_4 = [
    [['ㅇ','ㅑ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㄷ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    #-오 /EF
    [['ㅇ','ㅗ'],['ㅅ','ㅔ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['ㅇ','ㅔ','ㅇ','ㅛ']], #add jh_1116
    [['ㄴ','ㅔ'],['ㄴ','ㅔ','ㅇ','ㅛ']],
    
]
#/XSA, /EF
EF_AFTER_XSA_4 = [
    
    [['ㄷ','ㅏ'],['-special']],
    
    
]

VV_EF_4 = [
    [['ㄱ','ㅓ','ㄹ'],['ㄱ','ㅓ','ㄹ','ㅇ','ㅛ']],
    [['ㅎ','ㅐ','ㄹ','ㅏ'],['ㅎ','ㅏ','ㅅ','ㅔ','ㅇ','ㅛ']],
    [['ㅂ','ㅘ','ㄹ','ㅏ'],['ㅂ','ㅗ','ㅅ','ㅔ','ㅇ','ㅛ']],
    [['ㅂ','ㅘ'],['ㅂ','ㅘ','ㅇ','ㅛ']],
    [['ㅎ','ㅏ','ㄹ','ㄹ','ㅐ'],['ㅎ','ㅏ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ']],
    [['ㅈ','ㅜ','ㄹ','ㄹ','ㅐ'],['ㅈ','ㅜ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ']], #add jh_1115
    [['ㄷ','ㅙ'],['ㄷ','ㅙ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['-special']],
    [['ㄷ','ㅡ','ㄹ','ㄹ','ㅕ'],['ㄷ','ㅡ','ㄹ','ㄹ','ㅕ','ㅇ','ㅛ']],
    [['ㄷ','ㅏ','ㅁ','ㅕ','ㄴ','ㅅ','ㅓ'],['ㄷ','ㅏ','ㅁ','ㅕ','ㄴ','ㅅ','ㅓ','ㅇ','ㅛ']],
    [['ㄹ','ㅐ'],['ㄹ','ㅐ','ㅇ','ㅛ']],
    [['ㄴ','ㄷ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㅇ','ㅘ'],['ㅇ','ㅘ','ㅇ','ㅛ']], 
    [['ㅊ','ㅝ'],['ㅊ','ㅝ','ㅇ','ㅛ']],
    [['ㅔ'],['ㅔ','ㅇ','ㅛ']],
    [['ㅏ'],['ㅏ','ㅇ','ㅛ']], 
    [['ㅕ'],['ㅕ','ㅇ','ㅛ']],
    
]

##added code_JH
#/XSV+EF
XSV_EF_4 = [
    [['ㄷ','ㅏ'],['special']],
    [['ㅎ','ㅐ','ㄹ','ㅏ'],['ㅎ','ㅐ','ㅇ','ㅛ']],
    [['ㅎ','ㅐ'],['ㅎ','ㅐ','ㅇ','ㅛ']],
    [['ㄹ','ㄲ','ㅏ'],['ㄹ','ㄲ','ㅏ','ㅇ','ㅛ']],
    [['ㅏ','ㄴ','ㄷ','ㅏ'],['ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    
]
#/XSA+EF
XSA_EF_4 = [
    [['ㄷ','ㅏ'],['special']],
    [['ㄴ','ㄷ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㅎ','ㅏ','ㄴ','ㄷ','ㅔ'],['ㅎ','ㅏ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㅎ','ㅐ'],['ㅎ','ㅐ','ㅇ','ㅛ']],
    
]
#/VX+EF
VX_EF_4 = [
    [['ㄷ','ㅏ'],['special']],
    [['ㅎ','ㅐ'],['ㅎ','ㅐ','ㅇ','ㅛ']],
    [['ㄹ','ㅏ'],['ㅇ','ㅛ']],
    [['ㅂ','ㅘ','ㄹ','ㅏ'],['ㅂ','ㅗ','ㅅ','ㅔ','ㅇ','ㅛ']],
    [['ㅂ','ㅘ'],['ㅂ','ㅘ','ㅇ','ㅛ']],
    [['ㅈ','ㅜ','ㄹ','ㄹ','ㅐ'],['ㅈ','ㅜ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ']], #add jh_1115
    [['ㅁ','ㅏ'],['ㅁ','ㅏ','ㅇ','ㅛ']],
    [['ㄹ','ㄱ','ㅔ'],['ㄹ','ㄱ','ㅔ','ㅇ','ㅛ']],
    [['ㅝ'],['ㅝ','ㅇ','ㅛ']],
    [['ㄴ','ㄷ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    
    
]
#/NNB+JKO
NNB_JKO_4 = [
    [['ㄱ','ㅓ','ㄹ'],['specialx']],
]
#/EC
EC_4 = [
    [['ㄷ','ㅔ'],['specialx']],
    #[['ㄷ','ㅔ'],['ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㄱ','ㅔ'],['specialx']],
    [['ㄴ','ㅔ'],['specialx']],
    #[['ㄱ','ㅔ'],['ㄱ','ㅔ','ㅇ','ㅛ']],
]

#/VV
VV_4 = [
    
    #[['ㄷ','ㅚ'],['ㄷ','ㅚ','ㅇ','ㅛ']],
]
### Exception case dictionary ###
EXC_4_deal_1 = [
    
     ['ㄷㅏ',['ㄴ','ㅂㄴㅣㄷㅏ'],'ㅅㅡㅂㄴㅣㄷㅏ'],
]

EXC_4_deal_2 = [
    
    ['ㄴㅣ',['ㅆ','ㅇㅓㅇㅛ'],'ㅇㅛ']
]

EXC_4_deal_3 = [
    
    ['ㄴㅡㄴㄱㅏ',['ㅗ','ㅗ','ㅏㅇㅛ'],['ㅜ','ㅜ','ㅓㅇㅛ'],['ㅏ','ㅏ','ㅣㅇㅛ'],'ㅇㅓㅇㅛ'],
    ['ㄷㅏ',['ㅆ','ㅆ','ㅅㅡㅂㄴㅣㄷㅏ'],['ㄴ','ㅂ','ㄴㅣㄷㅏ'],['ㄹ','ㅂ','ㄴㅣㄷㅏ'],['ㅂ','ㅂ','ㅅㅡㅂㄴㅣㄷㅏ'],'ㅂㄴㅣㄷㅏ'],
    ['ㄴㅣ',['ㅆ','ㅆ','ㅇㅓㅇㅛ'],'ㅇㅛ']
]

EXC_4_deal_4 = [
    
    ['ㄱㅔ','ㄱㅔㅇㅛ',[['ㄵ','ㄵ','ㅇㅡㅅㅔㅇㅛ'],['ㄱ','ㄱ','ㅇㅡㅅㅔㅇㅛ'],['ㄷ','ㄷ','ㅇㅡㅅㅔㅇㅛ'],['ㅜㅂ','ㅜ','ㅇㅡㅅㅔㅇㅛ'],['ㅂ','ㅂ','ㅇㅡㅅㅔㅇㅛ'],'ㅅㅔㅇㅛ'],'ㄱㅔ'],
    ['ㄷㅔ','ㄷㅔㅇㅛ',[['','',''],''],'ㄷㅔ'],
    ['ㄴㅔ','ㄴㅔㅇㅛ',[['','',''],''],'ㄴㅔ'],
    ['ㄱㅓㄹ','ㄱㅓㄹㅇㅛ',[['','',''],''],'ㄱㅓㄹ']
]

#도록은 special

#### End of low -> high Dictionary####

#### Common Dictionary ####
#Yes or No
IC =[
    [['ㄴ','ㅔ'],['ㅇ','ㅡ','ㅇ']],
    [['ㅇ','ㅏ','ㄴ','ㅣ','ㅇ','ㅛ'],['ㅇ','ㅏ','ㄴ','ㅣ']],
    [['ㄴ','ㅔ'],['ㄱ','ㅡ','ㄹ','ㅐ']],
]


#### End of Common Dictionary ####
def unite(input, dict):
    for i in dict:
        input = re.sub(i[0],i[1],input)
    return input
    
## 자모 단위로 문장을 나누고 합칠 때 쓰는 class ##
class Jamodealer:
    jamo = []
    pp = ''
    #각 단어들을 받아와서 자모단위로 나눈다.
    def __init__(self,lis_word):
    
        self.jamo = []
        for i in lis_word:
            self.jamo.append(split_syllables(i))
    
    ##사전에서 변환된 자모단위로 분리된 문장을 합칠 때 쓰는 함수이다.     
    def make_one(self):
        #list 형태로 저장된 자모들의 집합을 하나의 string pp에 저장한다. 
        self.pp = ''
        for i in self.jamo:
             self.pp= self.pp+i
        ##종성과 종성을 합쳐야 하는 경우가 있다면 합친다.        
        self.pp = unite(self.pp, con_dict)
        
        #자모 단위의 string에서 자모 단위로 사전을 만들고 거기에 index를 부여한다.        
        chars = list(set(self.pp))
        char_to_ix = { ch:i for i,ch in enumerate(chars) }
        ix_to_char = { i:ch for i,ch in enumerate(chars) }
        
        #자모 단위로 분리되었던 문장을 다시 하나로 합친다.
        jamo_numbers = [char_to_ix[x] for x in self.pp]
        restored_jamo = ''.join([ix_to_char[x] for x in jamo_numbers])
        #합쳐진 문장을 return 한다.
        restored_text = join_jamos(restored_jamo)
        return restored_text
##End of Jamodealer class##

def tojamo(korean_word):
    r_lst = []
    for w in list(korean_word.strip()):
        ## 영어인 경우 구분해서 작성함. 
        if '가'<=w<='힣':
            ## 588개 마다 초성이 바뀜. 
            ch1 = (ord(w) - ord('가'))//588
            ## 중성은 총 28가지 종류
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append([CHOSUNG_LIST[ch1], JUNGSUNG_LIST[ch2], JONGSUNG_LIST[ch3]])
        else:
            r_lst.append([w])
    return r_lst

#Use in makestrdict
#Convert list to String
def li2str(input):
    st = ""
    for i in input:
        st = st+i
    return st

##Used when make dictionaries
def makestrdict(input):
    result = []
    for i in input:
        bullet = []
        for j in range(len(i)):
            gre = li2str(i[j])
            bullet.append(gre)
        result.append(bullet)
    return result

### *추가한 사전에 대한 str 사전을 생성, mapping 시키는 부분* ###

# * 기존의 방식은 다음과 같다. 어말어미는 어말어미 변환 방식 코드를 참고하면 된다.
# - 앞으로  사전을 추가하고 싶으면 여기서 코드를 추가하고 수정하면 된다.
# - dictionary를 만들고 해당 형태소에 해당하는 dictionary를 추가하면 된다.
#   예를 들어, EP(선어말어미)에 대한 사전을 만들고 싶다면 EP_dict = makestrdict(EP)를 선언하고
#   Dict_list에 'EP'를 추가하면 된다. 그 후 Mapping이 되도록 Dict_map에도 생성한 EP_dict를 추가하면 된다.

#EP_dict = makestrdict(EP)
EF_dict = makestrdict(EF)
NP_dict = makestrdict(NP)
JX_dict = makestrdict(JX)
VX_dict = makestrdict(VX)
VV_dict = makestrdict(VV)
XR_dict = makestrdict(XR)
NNG_dict = makestrdict(NNG)
NNP_dict = makestrdict(NNP)
UNKNOWN_dict = makestrdict(UNKNOWN)

Dict_list=['NNP','NP','JX','VX','VV','XR','NNG', 'UNKNOWN']

Map_family=[['V_fam',['VCP, VA, VX']], ['X_fam',['XSA','XSV']]]

Dict_map = [NNP_dict,NP_dict,JX_dict,VX_dict, VV_dict, XR_dict, NNG_dict, UNKNOWN_dict]

##현재의 어말어미 변환방식은 밑에 코드를 추가, 변경하면 된다. 
#high->low of string dictionary
EF_ONLY_dict = makestrdict(EF_ONLY)
EP_EF_dict = makestrdict(EP_EF)
VCP_EF_dict = makestrdict(VCP_EF)
A_EF_dict = makestrdict(A_EF)
XSA_EF_dict = makestrdict(XSA_EF)
XSV_EF_dict = makestrdict(XSV_EF)
JKB_JX_dict = makestrdict(JKB_JX)
VCN_EF_dict = makestrdict(VCN_EF)
EC_dict = makestrdict(EC)



#low->high of string dictionary
EF_ONLY_4S_dict = makestrdict(EF_ONLY_4S)
EF_ONLY_4C_dict = makestrdict(EF_ONLY_4C)
EP_EF_4_dict = makestrdict(EP_EF_4)
EF_AFTER_VCP_4_dict = makestrdict(EF_AFTER_VCP_4)
VCP_EF_4_dict = makestrdict(VCP_EF_4)
EF_AFTER_XSA_4_dict = makestrdict(EF_AFTER_XSA_4) ## add code_JM
VA_EF_4_dict = makestrdict(VA_EF_4)
VV_EF_4_dict = makestrdict(VV_EF_4)
VX_EF_4_dict = makestrdict(VX_EF_4)
XSV_EF_4_dict = makestrdict(XSV_EF_4)
NNB_JKO_4_dict = makestrdict(NNB_JKO_4)##add code_JH
EC_4_dict = makestrdict(EC_4)
VV_4_dict = makestrdict(VV_4)

##Use in Changer
def to2lists(input):
    lis_word = []
    lis_tag = []
    #data = han.pos(input,ntags=22,flatten=True, join=False)
    data = mec.pos(input)
    for i in data:
        lis_word.append(i[0])
        lis_tag.append(i[1])
    return lis_word, lis_tag

#add in 2021.09.26

#space의 index 찾는다.
#rememberSpace와 convertSpace는 문장의 space를 찾아 변환하고 그 위치에 다시 space를 넣기 위한 함수이다.
def rememberSpace(lis, input):
    
    rlis = []
    
    for i in range(len(lis)):
        if lis[i]==input:
            rlis.append(i)
            
    for i in range(len(rlis)):
        rlis[i] = rlis[i]-i      
    return rlis

def convertSpace(lis_space,lis_lis):
    
    rlis = []
    k=0
    for i in range(len(lis_lis)):
        
        if k in lis_space:
            rlis.append(i)
            
        k = k+len(lis_lis[i])
        
    #print(rlis)  
    return rlis

def union(lis, lis_lis):
    
    k = 0
    for i in lis:
        lis_lis.insert(i+k,' ')
        k = k+1
        
##특정 형태소를 찾는다. 종결어미(EF)를 찾아서 높임말 반말 변환 시 사용된다.         
def findTargetMorphology(input, word):
    
    i=0
    lis = []
    for j in range(len(input)):
        if word in input[j]:
            lis.append(j)
            
    return lis

def del_over(input):
    
    res = set(input) #집합set으로 변환
    result = list(res) #list로 변환
    return result


## Main Class ## 
class Changer(object):
    
    ## High -> Low ##    
    def high_low(self, stc):
        result = stc
        
        space_list = rememberSpace(stc,' ')
        
        lis_word, lis_tag = to2lists(result)
        space_location = convertSpace(space_list, lis_word)
        jam = Jamodealer(lis_word)
        lis = []
        key = -1
        for i in H_LIST:
            if i in lis_word[-2]:
                key = 1
        #if key>0:
        if key>0 or key<0:     
            for i in range(len(lis_tag)):
                res = jam.jamo[i]
                dic = []
                if 'EF' in lis_tag[i]:
                    if 'EF' == lis_tag[i]:
                        dic = EF_ONLY_dict
                    elif 'EP' in lis_tag[i]:
                        dic = EP_EF_dict
                    elif 'VCP' in lis_tag[i]:
                        dic = VCP_EF_dict
                    elif 'XSA' in lis_tag[i]:
                        dic = XSA_EF_dict
                    elif 'XSV' in lis_tag[i]:
                        dic = XSV_EF_dict
                    elif 'VCN' in lis_tag[i]:
                        dic = VCN_EF_dict
                    else:
                        dic = A_EF_dict
                elif 'JX' in lis_tag[i]:
                    if 'JKB' in lis_tag[i-1]:
                        dic = JKB_JX_dict
                    else:
                        dic = JX_dict
                elif 'EC' in lis_tag[i]:
                    dic = EC_dict
                        
                #else Dict_list[k] in lis_tag[i]:
                else:
                    for k in range(len(Dict_list)):
                        if Dict_list[k] in lis_tag[i]:
                            dic = Dict_map[Dict_list.index(Dict_list[k])]
                            break
                        
                flag = 0    
                for j in range(len(dic)):
                    if dic[j][0] in res:
                        flag = 1
                        if self.isExcept(dic[j])==1:
                            for q in range(len(EXC_deal_1)):
                                if dic[j][0]==EXC_deal_1[q][0]:
                                    for r in range(1,len(EXC_deal_1[q])):
                                        
                                        l = len(EXC_deal_1[q][r][0])
                                        l = l*(-1)
                                        
                                        if jam.jamo[i-1][l:]==EXC_deal_1[q][r][0]:
                                            res = re.sub(dic[j][0],EXC_deal_1[q][r][1],res)
                                        
                        elif self.isExcept(dic[j])==2:
                            for q in range(len(EXC_deal_2)):
                                if dic[j][0]==EXC_deal_2[q][0]:
                                    if i+1 in space_location:
                                        res = re.sub(dic[j][0], EXC_deal_2[q][2], res)
                                    else:
                                        res = re.sub(dic[j][0], EXC_deal_2[q][1], res)
                                    
                        elif self.isExcept(dic[j])==3:
                            for q in range(len(EXC_deal_3)):
                                if dic[j][0]==EXC_deal_3[q][0]:
                                    for o in range(len(EXC_deal_3[q][1])):
                                        
                                        l = len(EXC_deal_3[q][1][o][0])
                                        l = l*(-1)
                                        
                                        if jam.jamo[i-1][l:]==EXC_deal_3[q][1][o][0]:
                                            res = re.sub(dic[j][0],EXC_deal_3[q][1][o][1],res)
                                        else:
                                            if o==len(EXC_deal_3[q][1])-1:
                                                res = re.sub(dic[j][0],EXC_deal_3[q][2],res)
                                                
                        
                        elif self.isExcept(dic[j])==4:
                            for q in range(len(EXC_deal_4)):
                                if dic[j][0]==EXC_deal_4[q][0]:
                                    if lis_tag[i+1] =='SF':
                                        #if lis_tag[i-1]=='JKB':
                                        flag_1=0
                                        for t in range(len(EXC_deal_4[q][2])-1):
                                            if flag_1==1:
                                                break
                                                    
                                                
                                            if EXC_deal_4[q][2][t][0] in jam.jamo[i-1]:
                                                if len(EXC_deal_4[q][2][t][0])>1:
                                                    li_1 = list(jam.jamo[i-1])
                                                    li_1=li_1[:-len(EXC_deal_4[q][2][t][0])]
                                                    li_1.append(EXC_deal_4[q][2][t][1])
                                                    lis[i-1] = ''.join(li_1)

                                                    #jam.jamo[ind-1] = re.sub(EXC_4_deal_4[q][2][t][0],EXC_4_deal_4[q][2][1],jam.jamo[ind-1])
                                                    res = re.sub(dic[j][0], EXC_deal_4[q][2][t][2],res)
                                                    flag_1=1
                                                        
                                                else:
                                                    li_1 = list(jam.jamo[i-1])
                                                    li_1[-1] = EXC_deal_4[q][2][t][1]
                                                    jam.jamo[i-1] = ''.join(li_1)

                                                    #jam.jamo[ind-1] = re.sub(EXC_4_deal_4[q][2][t][0],EXC_4_deal_4[q][2][1],jam.jamo[ind-1])
                                                    res = re.sub(dic[j][0], EXC_deal_4[q][2][t][2],res)
                                                    flag_1=1
                                                        
                                        if flag_1==0:
                                            res = re.sub(dic[j][0],EXC_deal_4[q][2][-1],res)
                                        #else:
                                        #    res = re.sub(dic[j][0],EXC_deal_4[q][1],res)
                                    else:
                                        res = re.sub(dic[j][0],EXC_deal_4[q][-1],res)
                        
                                    
                        else:
                            res = re.sub(dic[j][0],dic[j][1],res)
                            
                        if flag==1:
                            break
                            
                        #jam.jamo[i] = res
                lis.append(res)
            
            
            #print(jam.jamo[i])
        
        union(space_location, lis)
        jam.jamo = []
        for i in range(len(lis)):
            jam.jamo.append(lis[i])
            #print(lis[i])
        
        #union(space_location, jam.jamo)
        return jam.make_one()
    
    def low_high(self, stc):
        result = stc
        
        space_list = rememberSpace(stc, ' ')
        
        lis_word, lis_tag = to2lists(result)
        space_location = convertSpace(space_list, lis_word)
        jam = Jamodealer(lis_word)
        #print(jam.jamo)
        lis = []
        
        ## 종결어미만을 변경
        ind_trans_ef = findTargetMorphology(lis_tag, 'EF')
        ind_trans_jko = findTargetMorphology(lis_tag, 'JKO')
        ind_trans_ec = findTargetMorphology(lis_tag, 'EC')
        ind_trans_vv = findTargetMorphology(lis_tag, 'VV')
        
        ind_union = ind_trans_ef + ind_trans_jko + ind_trans_ec + ind_trans_vv
        ind_union = del_over(ind_union)
        
        for ind in ind_union:  #modified_2021.10.31
            key = -1

            for i in H_LIST:
                if i not in lis_word[ind]:
                    key = 1
                else:
                    key = -1
                if key==-1:
                    break
            if key >0:
                res = jam.jamo[ind]
                dic = []
                #EF
                if 'EF' in lis_tag[ind]:

                    if 'EF' == lis_tag[ind]:
                        if '+' in lis_tag[ind-1]:
                            dic = EF_ONLY_4C_dict
                        elif lis_tag[ind-1]=='VCP':
                            dic = EF_AFTER_VCP_4_dict
                        elif lis_tag[ind-1]=='XSA':
                            dic = EF_AFTER_XSA_4_dict
                        else:
                            dic = EF_ONLY_4S_dict
                            #print('ee')

                    elif 'EP' in lis_tag[ind]:
                        dic = EP_EF_4_dict
                    elif 'VCP' in lis_tag[ind]:
                        dic = VCP_EF_4_dict
                    elif 'VA' in lis_tag[ind]:
                        dic = VA_EF_4_dict
                    elif 'VV' in lis_tag[ind]:
                        dic = VV_EF_4_dict
                    elif 'VX' in lis_tag[ind]:
                        dic = VX_EF_4_dict
                    elif 'XSV' in lis_tag[ind]:
                        dic = XSV_EF_4_dict
                    else:
                        dic = A_EF_dict
                #JKO      
                elif 'JKO' in lis_tag[ind]:
                    dic = NNB_JKO_4_dict
                    

                elif 'EC' in lis_tag[ind]:
                    dic = EC_4_dict
            
                ##added by JM
                flag = 0        
                for j in range(len(dic)):
                    if dic[j][0] in res:
                        flag = 1
                        
                        if self.isExcept(dic[j])==1:
                            for q in range(len(EXC_4_deal_1)):
                                if dic[j][0]==EXC_4_deal_1[q][0]:
                                    flag_1=0
                                    for r in range(1,len(EXC_4_deal_1[q])-1):
                                        if flag_1==1:
                                            break
                                        if EXC_4_deal_1[q][r][0] in jam.jamo[ind]:
                                            res = re.sub(EXC_4_deal_1[q][r][0]+dic[j][0],EXC_4_deal_1[q][r][1],res)
                                            flag_1=1
                                    if flag_1==0:
                                        res = re.sub(dic[j][0],EXC_4_deal_1[q][-1],res)

                        elif self.isExcept(dic[j])==2:
                            for q in range(len(EXC_4_deal_2)):
                                if dic[j][0]==EXC_4_deal_2[q][0]:
                                    if ind+1 in space_location:
                                        res = re.sub(dic[j][0], EXC_4_deal_2[q][2], res)
                                    else:
                                        res = re.sub(dic[j][0], EXC_4_deal_2[q][1], res)
                                        
                        elif self.isExcept(dic[j])==3:
                            for q in range(len(EXC_4_deal_3)):
                                if dic[j][0]==EXC_4_deal_3[q][0]:
                                    flag_1 = 0
                                    for r in range(1,len(EXC_4_deal_3[q])-1):
                                        if flag_1==1:
                                            break
                                        if jam.jamo[ind-1][-1]==EXC_4_deal_3[q][r][0]:
                                            flag_1=1
                                            li_1 = list(jam.jamo[ind-1])
                                            li_1[-1] = EXC_4_deal_3[q][r][1]
                                            jam.jamo[ind-1] = ''.join(li_1)
                                            res = re.sub(dic[j][0],EXC_4_deal_3[q][r][2],res)
                                    if flag_1==0:
                                            res = re.sub(dic[j][0],EXC_4_deal_3[q][-1],res)
                                            
                        elif self.isExcept(dic[j])==4:
                            for q in range(len(EXC_4_deal_4)):
                                if dic[j][0]==EXC_4_deal_4[q][0]:
                                    if lis_tag[ind+1] =='SF':
                                        if lis_tag[ind-1]=='VV':
                                            flag_1=0
                                            for t in range(len(EXC_4_deal_4[q][2])-1):
                                                if flag_1==1:
                                                    break
                                                    
                                                
                                                    
                                                if EXC_4_deal_4[q][2][t][0] in jam.jamo[ind-1]:
                                                    if len(EXC_4_deal_4[q][2][t][0])>1:
                                                        li_1 = list(jam.jamo[ind-1])
                                                        li_1=li_1[:-len(EXC_4_deal_4[q][2][t][0])]
                                                        li_1.append(EXC_4_deal_4[q][2][t][1])
                                                        jam.jamo[ind-1] = ''.join(li_1)

                                                        #jam.jamo[ind-1] = re.sub(EXC_4_deal_4[q][2][t][0],EXC_4_deal_4[q][2][1],jam.jamo[ind-1])
                                                        res = re.sub(dic[j][0], EXC_4_deal_4[q][2][t][2],res)
                                                        flag_1=1
                                                        
                                                    else:
                                                        li_1 = list(jam.jamo[ind-1])
                                                        li_1[-1] = EXC_4_deal_4[q][2][t][1]
                                                        jam.jamo[ind-1] = ''.join(li_1)

                                                        #jam.jamo[ind-1] = re.sub(EXC_4_deal_4[q][2][t][0],EXC_4_deal_4[q][2][1],jam.jamo[ind-1])
                                                        res = re.sub(dic[j][0], EXC_4_deal_4[q][2][t][2],res)
                                                        flag_1=1
                                                    
                                            if flag_1==0:
                                                res = re.sub(dic[j][0],EXC_4_deal_4[q][2][-1],res)
                                        else:
                                            res = re.sub(dic[j][0],EXC_4_deal_4[q][1],res)
                                    else:
                                        res = re.sub(dic[j][0],EXC_4_deal_4[q][-1],res)
                    
                        else:
                            res = re.sub(dic[j][0],dic[j][1],res)

                        
                        if flag==1:
                            break
                        
                jam.jamo[ind] = res


            
        
        
        union(space_location, jam.jamo)

        return jam.make_one()
            
          
    def isExcept(self, input):
        if input[1]=='special':
            return 1
        elif input[1]=='special-':
            return 2
        elif input[1]=='-special':
            return 3
        elif input[1]=='specialx':
            return 4
        else:
            return 0
        
    def indicator(self, ind, lis, tag, ex_word, exc_tags):
        re = 1
        for j in range(len(ex_word)):
            if exc_tags[j] in tag[ind+1]:
                for i in range(len(ex_word)):
                    if ex_word[i][0] in lis[ind+1]:
                       
                        re = ex_word[i][1]
                        break
                        
                        print(lis[ind-1])
        return re
        
        
#     def processText(self,stc):
#         result = stc
#         res = self.high_low(result)
#         #spacing = Spacing()
#         #res = spacing(res)
#         return res
#     ##change function name
#     def processText_convertHigh(self,stc):
#         result = stc
#         res = self.low_high(result)
#         return res
    def processText(self,stc):
        result = stc
        
        flag = 0
        if result[-1]=='\n':
            result = result.replace('\n','')
        if result[-1] =='?' or result[-1] =='.' or result[-1] =='!' or result[-1] =='\"':
            result = result
        else:
            result = result+'.'
            flag = 1
        
        res = self.high_low(result)
        
        if flag ==1:
            res = res[:-1]
        
        #spacing = Spacing()
        #res = spacing(res)
        return res
    
    def processText_convertHigh(self,stc):
        result = stc
        
        flag = 0
        if result[-1]=='\n':
            result = result.replace('\n','')
        if result[-1] =='?' or result[-1] =='.' or result[-1] =='!' or result[-1] =='\"':
            result = result
        else:
            result = result+'.'
            flag = 1
        
        res = self.low_high(result)
        
        if flag ==1:
            res = res[:-1]
        return res

In [2]:
#txt = '전 그것이 맞다고 생각합니다.'
tx = '각각 위에서부터 변환함수를 위한 함수, 사전, 변환 함수의 일부다. 모두 한 파일에 정리되어 있기에 깔끔하지 못하며, 각 부분을 간단한 주석처리로 구분해 놓았기에 직관성이 몹시 떨어지는 요소였다.'
ch = Changer()
#tt = ch.processText(tx)
ttt = ch.processText_convertHigh(tx)
#print(tt)
print(ttt)

각각 위에서부터 변환함수를 위한 함수, 사전, 변환 함수의 일붑니다. 모두 한 파일에 정리되어 있기에 깔끔하지 못하며, 각 부분을 간단한 주석처리로 구분해 놓았기에 직관성이 몹시 떨어지는 요소였습니다.


In [5]:
#txt = '전 그것이 맞다고 생각합니다.'
tt = '그를 부르세요.'

#tt = ch.processText(tx)
ttt = ch.processText(tt)
#print(tt)
print(ttt)

그를 불러.


In [4]:
f0 = open('./hgu_clean.kr', 'r', encoding = 'utf-8')
f1 = open('./1st_dis_01', 'w',encoding = 'utf-8')
f2 = open('./2nd_dis_01', 'w',encoding = 'utf-8')
f3 = open('./3rd_dis_01', 'w',encoding = 'utf-8')
flag = 0

num = 0
number = 0
ch = Changer()

while True:
    flag = 0
    line = f0.readline()
    if not line: break
    if line[-1]=='\n':
        line = line.replace('\n','')
    
    cc = ch.processText(line)
    
    if number<300000:
        if line!=cc:
            f1.write(line)
            f1.write('\n')
            f1.write(cc)
            f1.write('\n\n')
        num = num+1
    elif number>600000:
        if line!=cc:
            f2.write(line)
            f2.write('\n')
            f2.write(cc)
            f2.write('\n\n')
    else:
        if line!=cc:
            f3.write(line)
            f3.write('\n')
            f3.write(cc)
            f3.write('\n\n')
        
        
    number = number+1
f0.close()
f1.close()
f2.close()
f3.close()
print(number)
print(num)

KeyboardInterrupt: 